In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

!pip install fuzzywuzzy
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

     |████████████████████████████████| 50 kB 2.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=3d4a218f0502147d5c3c8903ff4d7209a109a30852969277396527424a13ad4e
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
df = pd.read_csv("Total_Managers_pos.csv")
print(df)

                                                filename  \
0       2005-Jan-27-SHLDQ.PK-137041167100-Transcript.txt   
1       2005-Jan-27-SHLDQ.PK-137041167100-Transcript.txt   
2       2005-Jan-27-SHLDQ.PK-137041167100-Transcript.txt   
3       2005-Jan-27-SHLDQ.PK-137041167100-Transcript.txt   
4       2005-Jan-27-SHLDQ.PK-137041167100-Transcript.txt   
...                                                  ...   
709610  2017-Aug-24-SHLDQ.PK-139913387379-Transcript.txt   
709611  2017-Aug-24-SHLDQ.PK-139913387379-Transcript.txt   
709612  2017-Aug-24-SHLDQ.PK-139913387379-Transcript.txt   
709613  2017-Aug-24-SHLDQ.PK-139913387379-Transcript.txt   
709614     2017-Aug-24-SIG.N-137535371251-transcript.txt   

                                         filename_manager            manager  \
0       2005-Jan-27-SHLDQ.PK-137041167100-Transcript-G...      Glenn Richter   
1       2005-Jan-27-SHLDQ.PK-137041167100-Transcript-G...      Glenn Richter   
2       2005-Jan-27-SHLDQ.PK-1370411671

In [ ]:
df['position'] = df['position'].map(lambda position: str(position))
df['position'] = df['position'].map(lambda position: position.strip().title().replace('.',''))

In [ ]:
CFO_pure = ['Chief Financial Officer', 'Cfo', 'Chief Financial Officer,', 'Chief Finacial Officer', 'Chief Financia Officer', 'Chief Fiancial Officer', 'Chief Fianncial Officer', 'Cheif Financial Officer', 'Chief Finanical Officer', 'Chief Financial Office', 'Chief Financial', 'Chief Finance Officer', 'Cheif Financial Officer', 'Chief Executive Financial Officer', 'CFO', 'Andcfo', 'Interimcfo']
CFO_subs = ['Acting Chief Financial Officer', 'Interim Chief Financial Officer', 'Interim Financial Officer', 'Co-Chief Financial Officer', 'Deputy Chief Financial Officer', 'Group Chief Financial Officer', 'Future Chief Financial Officer', 'Financial Officer', 'Finance Officer', 'Acfo']
CFO_overlap = ['Chief Financial Officer', 'Chief Financial Officer,', 'Cheif Financial Officer', 'Chief Finance Officer', 'Chief Executive Financial Officer']
CFO_combo_spell = ['Senior Vice President & Chief Financial Offier', 'Chief Financial Offiicer & Chief Corporate Officer', 'Chief Financial  Officer And Senior Vice President']

In [ ]:
def CFO_subs_indexing(positions):
  if positions['position'] == 'Acting Chief Financial Officer' or positions['position'] == 'Interim Chief Financial Officer' or positions['position'] == 'Interim Financial Officer' or positions['position'] == 'Co-Chief Financial Officer' or positions['position'] == 'Deputy Chief Financial Officer' or positions['position'] == 'Group Chief Financial Officer' or positions['position'] == 'Future Chief Financial Officer' or positions['position'] == 'Financial Officer' or positions['position'] == 'Finance Officer':
    return 1
  elif 'Acting Chief Financial Officer' in positions['position'] or 'Interim Chief Financial Officer' in positions['position'] or 'Interim Financial Officer' in positions['position'] or 'Co-Chief Financial Officer' in positions['position'] or 'Deputy Chief Financial Officer' in positions['position'] or 'Group Chief Financial Officer' in positions['position'] or 'Future Chief Financial Officer' in positions['position'] or 'Interimcfo' in positions['position'] or 'Acfo' in positions['position'] or 'Cfo-Elect' in positions['position'] or 'Co-Cfo' in positions['position']:
    return 1
  elif 'Cfo of' in positions['position']:
    return 0
  else:
    return 0

In [ ]:
df['CFO_alt_index'] = df.apply(lambda positions: CFO_subs_indexing(positions), axis = 1)

In [ ]:
def CFO_indexing(positions):
  if positions['position'] == 'Chief Financial Officer' or positions['position'] == 'Cfo' or positions['position'] == 'Chief Financial Officer,' or positions['position'] == 'Chief Finacial Officer' or positions['position'] == 'Chief Financia Officer' or positions['position'] == 'Chief Fiancial Officer' or positions['position'] == 'Chief Fianncial Officer' or positions['position'] == 'Cheif Financial Officer' or positions['position'] == 'Chief Finanical Officer' or positions['position'] == 'Chief Financial Office' or positions['position'] == 'Chief Financial' or positions['position'] == 'Chief Finance Officer' or positions['position'] == 'Chief Executive Financial Officer':
    return 1
  elif ('Cfo Of' in positions['position']) and ('Cfo And' not in positions['position']) and ('Cfo,' not in positions['position']) and ('Cfo &' not in positions['position']):
    return 0
  elif (positions['CFO_alt_index'] == 0) and ('Chief Financial Officer' in positions['position'] or 'Chief Financil Officer' in positions['position'] or "Cfo" in positions['position'] or 'Chief Financial Officer,' in positions['position'] or 'Chief Finacial Officer' in positions['position'] or 'Chief Financia Officer' in positions['position'] or 'Chief Fiancial Officer' in positions['position'] or 'Chief Fianncial Officer' in positions['position'] or 'Cheif Financial Officer' in positions['position'] or 'Chief Finanical Officer' in positions['position'] or 'Chief Financial Office' in positions['position'] or 'Chief Financial' in positions['position'] or 'Chief Finance' in positions['position'] or 'Chief Finance Officer' in positions['position'] or 'Chief Executive Financial Officer' in positions['position'] or 'CFO' in positions['position'] or 'Andcfo' in positions['position'] or 'Chief Strategy & Financial' in positions['position']): #and ('Cfo Of' not in positions['position']):
    return 1
  else:
    return 0

In [ ]:
df['CFO_index'] = df.apply(lambda positions: CFO_indexing(positions), axis = 1)

TRIAL CODE

In [ ]:
def CFO_indexing(positions):
  if positions['position'] == [pure_word for pure_word in CFO_pure]:
    return 1
  elif [pure_word in positions['position'] for pure_word in CFO_pure] and positions['CFO_alt_index'] == 0:
    return 1
  elif [combo_word in positions['position'] for combo_word in CFO_combo_spell]:
    return 1
  else:
    return 0

In [ ]:
def CFO_subs_indexing(positions):
  for sub_word in CFO_subs:
    if positions['position'] == sub_word:
      return 1
    elif positions['position'] == [pure_word for pure_word in CFO_pure]:
      return 0
  #elif [sub_word in positions['position'] for sub_word in CFO_subs] == True and [overlap_word not in positions['position'] for overlap_word in CFO_overlap] == True:
    #return 1
  else:
    return 0

CEO INDEXING

In [ ]:
CEO_pure = ['Chief Executive Officer', 'Ceo', 'Chief Executive Officer,', 'Cheif Executive Officer', 'Chief Executive Offcier', 'Chief Executive', 'Chief Executive Office', 'Chief Executive Financial Officer', 'Chief Eexecutive Officer', 'Chairmanceo', 'Andceo', 'Cceo', 'Chief Exeucutive Officer']
CEO_subs = ['Acting Chief Executive Officer', 'Interim Chief Executive Officer', 'Co-Chief Executive Officer', 'Executive Officer', 'Coceo']
CEO_overlap = ['Chairman And Chief Excutive Officer', 'President, Chief Exeucutive Officer', 'Chairman And Chief Excutive Officer', 'Chairman And Chief Eexcutive Officer']

In [ ]:
def CEO_subs_indexing(positions):
  if positions['position'] == 'Acting Chief Executive Officer' or positions['position'] == 'Interim Chief Executive Officer' or positions['position'] == 'Co-Chief Executive Officer' or positions['position'] == 'Executive Officer' or positions['position'] == 'Coceo':
    return 1
  elif 'Acting Chief Executive Officer' in positions['position'] or 'Interim Chief Executive Officer' in positions['position'] or 'Co-Chief Executive Officer' in positions['position'] or 'Coceo' in positions['position'] or 'Co-Ceo' in positions['position'] or 'Aceo' in positions['position'] or 'Ceo-Elect' in positions['position']:
    return 1
  elif 'Ceo of' in positions['position'] or 'Chief Executive Officer of' in positions['position']:
    return 0
  else:
    return 0

In [ ]:
df['CEO_alt_index'] = df.apply(lambda positions: CEO_subs_indexing(positions), axis = 1)

In [ ]:
def CEO_indexing(positions):
  if (positions['position'] == 'Chief Executive Officer' or positions['position'] == 'Ceo' or positions['position'] == 'Chief Executive Officer' or positions['position'] == 'Cheif Executive Officer' or positions['position'] == 'Chief Executive Offcier' or positions['position'] == 'Chief Executive' or positions['position'] == 'Chief Executive Office' or positions['position'] == 'Chief Executive Financial Officer' or positions['position'] == 'Chief Eexecutive Officer' or positions['position'] == 'Chairmanceo' or positions['position'] == 'Cceo' or positions['position'] == 'Chief Exeucutive Officer') and ('Ceo of' not in positions['position'] or 'Chief Executive Officer of' not in positions['position']):
    return 1
  elif ('Ceo Of' in positions['position'] or 'Chief Executive Officer Of' in positions['position']) and ('Ceo Director' not in positions['position']) and ('Ceo And' not in positions['position']) and ('Ceo,' not in positions['position']) and ('Ceo &' not in positions['position']):
    return 0
  elif (positions['CEO_alt_index'] == 0) and ('Chief Executive Officer' in positions['position'] or 'Ceo' in positions['position'] or 'Chief Executive Officer,' in positions['position'] or 'Cheif Executive Officer' in positions['position'] or 'Chief Executive Offcier' in positions['position'] or 'Chief Executive' in positions['position'] or 'Chief Executive Office' in positions['position'] or 'Chief Executive Financial Officer' in positions['position'] or 'Chief Eexecutive Officer' in positions['position'] or 'Chairmanceo' in positions['position'] or 'Andceo' in positions['position'] or 'Cceo' in positions['position']  or 'Chief Exeucutive Officer' in positions['position'] or 'Chief Excutive Officer' in positions['position'] or 'Chief Eexcutive Officer' in positions['position']): #and ('Ceo Of' not in positions['position'] and 'Chief Executive Officer Of' not in positions['position']):
    return 1
  else:
    return 0

In [ ]:
df['CEO_index'] = df.apply(lambda positions: CEO_indexing(positions), axis = 1)

In [ ]:
df.to_csv('Total_Managers_pos_Indexed_R1.csv')